In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 
import joblib

%matplotlib inline
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv("/kaggle/input/tabular-playground-series-jan-2021/train.csv", index_col=["id"])
test_df = pd.read_csv("/kaggle/input/tabular-playground-series-jan-2021/test.csv", index_col=["id"])

X = train_df.iloc[:, :-1].to_numpy()
y = train_df.iloc[:, -1].to_numpy()
X_test = test_df.to_numpy()

In [ ]:
corr=train_df.corr()["target"]
corr[np.argsort(corr, axis=0)[:-1]]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import AdaBoostRegressor,  HistGradientBoostingRegressor, StackingRegressor, RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import SGDRegressor

import xgboost as xgb

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

import optuna 
from optuna import Trial, visualization
from optuna.samplers import TPESampler

# optuna.logging.set_verbosity(optuna.logging.WARNING)

def train(model):
    X_train, X_test, y_train, y_test = train_test_split(X, y.flatten(), test_size=0.1, random_state=156)
    y_train = y_train.reshape(-1, 1)
    y_test  = y_test.reshape(-1, 1)
        
    model = model.fit(X_train, y_train, early_stopping_rounds=100, verbose=False, eval_set=[(X_test, y_test)])
    score = mean_squared_error(model.predict(X_train), y_train, squared=False)
    print(score)
    return model

Integer parameter : A uniform distribution on integers.  
```n_estimators = trial.suggest_int('n_estimators',100,500)```

Categorical parameter : A categorical distribution.  
```criterion = trial.suggest_categorical('criterion' ,['gini', 'entropy'])```

Uniform parameter : A uniform distribution in linear domain.  
```subsample = trial.suggest_uniform('subsample' ,0.2,0.8)```

Discrete-uniform parameter : A discretized uniform distribution in linear domain.  
```max_features = trial.suggest_discrete_uniform('max_features', 0.05,1,0.05)```

Loguniform parameter : A uniform distribution in log domain.  **
```learning_rate = trial.sugget_loguniform('learning_rate' : 1e-6, 1e-3)```

In [ ]:
def objectiveXGB(trial: Trial, X, y, test):
    param = {
        "n_estimators" : trial.suggest_int('n_estimators', 500, 4000),
        'max_depth':trial.suggest_int('max_depth', 8, 16),
        'min_child_weight':trial.suggest_int('min_child_weight', 1, 300),
        'gamma':trial.suggest_int('gamma', 1, 3),
        'learning_rate': 0.01,
        'colsample_bytree':trial.suggest_discrete_uniform('colsample_bytree',0.5, 1, 0.1),
        'nthread' : -1,
        'tree_method': 'gpu_hist',
        'predictor': 'gpu_predictor',
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'subsample': trial.suggest_categorical('subsample', [0.6,0.7,0.8,1.0] ),
        'random_state': 42
    }
    X_train, X_test, y_train, y_test = train_test_split(X, y.flatten(), test_size=0.1)
    
    y_train = y_train.reshape(-1, 1)
    y_test  = y_test.reshape(-1, 1)

    model = xgb.XGBRegressor(**param)
    xgb_model = model.fit(X_train, y_train, verbose=False, eval_set=[(X_test, y_test)])
    score = mean_squared_error(xgb_model.predict(X_test), y_test, squared=False)

    return score

In [ ]:
study = optuna.create_study(direction='minimize',sampler=TPESampler())
study.optimize(lambda trial : objectiveXGB(trial, X,  y, X_test), n_trials=50)
print('Best trial: score {},\nparams {}'.format(study.best_trial.value,study.best_trial.params))

best_param = study.best_trial.params
xgbReg = train(xgb.XGBRegressor(**best_param, tree_method='gpu_hist', random_state=42, predictor='gpu_predictor', learning_rate=0.01, nthread=-1))

#params =  {'n_estimators': 3520, 'max_depth': 11, 'min_child_weight': 231, 'gamma': 2, 'colsample_bytree': 0.7, 'lambda': 0.014950936465569798, 'alpha': 0.28520156840812494, 'subsample': 0.6}
#xgbReg = train(xgb.XGBRegressor(**params, tree_method='gpu_hist', random_state=42, predictor='gpu_predictor', learning_rate=0.01, nthread=-1))

# 0.6744648190960726

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_param_importances(study)

In [ ]:
def objectiveLGBM(trial: Trial, X, y, test):
    param = {
        'objective': 'regression',
        'metric': 'root_mean_squared_error',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 512),
        'learning_rate': 0.01,
        'n_estimators': trial.suggest_int('n_estimators', 700, 3000),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'device':"gpu",
        'gpu_use_dp':True
    }
    X_train, X_test, y_train, y_test = train_test_split(X, y.flatten(), test_size=0.1)
        
    lgbm_regr = LGBMRegressor(**param)
    lgbm_regr = lgbm_regr.fit(X_train, y_train, verbose=False)
    
    score = mean_squared_error(lgbm_regr.predict(X_test), y_test, squared=False)
    return score

In [ ]:
study = optuna.create_study(direction='minimize',sampler=TPESampler())
study.optimize(lambda trial : objectiveLGBM(trial, X, y, X_test), n_trials=20)
print('Best trial: score {},\nparams {}'.format(study.best_trial.value,study.best_trial.params))

best_param2 = study.best_trial.params
lgbm = LGBMRegressor(**best_param2, device="gpu",gpu_use_dp=True, objective='regression', metric='root_mean_squared_error',  learning_rate= 0.01, boosting_type='gbdt')

# Best trial: score 0.6934602592622415,
# params {'lambda_l1': 4.168316306871167e-05, 'lambda_l2': 1.1288557039193647e-05, 'num_leaves': 98, 'n_estimators': 2280, 'feature_fraction': 0.7977209715952681, 'bagging_fraction': 0.4353577523638581, 'bagging_freq': 4, 'min_child_samples': 69}

In [ ]:
from vecstack import stacking

final_model = xgb.XGBRegressor(n_estimators= 2000, max_depth= 16,tree_method='gpu_hist', predictor='gpu_predictor')
sgd = SGDRegressor(max_iter=1000)
hgb = HistGradientBoostingRegressor( max_depth=3, min_samples_leaf=1)
cat = CatBoostRegressor(task_type="GPU", verbose=False)

estimators = [
    lgbm, cat, sgd, hgb, xgbReg
]

S_train, S_test = stacking(estimators,  X, y, X_test, regression=True, metric=mean_squared_error, n_folds=5, 
                           shuffle=False, random_state=0, verbose=2)
final_model.fit(S_train, y)

print(mean_squared_error(final_model.predict(S_train), y, squared=False))

In [ ]:
submission = pd.read_csv("/kaggle/input/tabular-playground-series-jan-2021/test.csv", index_col=["id"])
y_hat = final_model.predict(S_test)        

submission["target"] = y_hat 
submission[["target"]].to_csv("/kaggle/working/submission_stacking.csv")
joblib.dump(final_model, '/kaggle/working/skacking.pkl')

In [ ]:
submission = pd.read_csv("/kaggle/input/tabular-playground-series-jan-2021/test.csv", index_col=["id"])
lgbm = LGBMRegressor(**best_param2, device="gpu",gpu_use_dp=True, objective='regression', learning_rate= 0.01, metric='root_mean_squared_error', boosting_type='gbdt')
lgbm = lgbm.fit(X, y, verbose=False)
y_hat = lgbm.predict(submission.to_numpy())        
print(mean_squared_error(lgbm.predict(X), y, squared=False))
submission["target"] = y_hat 
submission[["target"]].to_csv("/kaggle/working/submission_lgbm.csv")
joblib.dump(lgbm, '/kaggle/working/lgbm.pkl')

In [ ]:
submission = pd.read_csv("/kaggle/input/tabular-playground-series-jan-2021/test.csv", index_col=["id"])
params =  {'n_estimators': 3520, 'max_depth': 11, 'min_child_weight': 231, 'gamma': 2, 'colsample_bytree': 0.7, 'lambda': 0.014950936465569798, 'alpha': 0.28520156840812494, 'subsample': 0.6}
xgbReg = train(xgb.XGBRegressor(**params, tree_method='gpu_hist', random_state=42, predictor='gpu_predictor', learning_rate=0.01, nthread=-1))

y_hat = xgbReg.predict(submission.to_numpy())        
print(mean_squared_error(xgbReg.predict(X), y, squared=False))

submission["target"] = y_hat 
submission[["target"]].to_csv("/kaggle/working/submission_xgb.csv")
joblib.dump(xgbReg, '/kaggle/working/xgb_reg.pkl')